<a href="https://colab.research.google.com/github/WilliamAshbee/splineexample/blob/main/SimplePointUber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url


In [20]:
class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    self.input_dim = 3*1024

    self.layers = nn.Sequential(
        nn.Linear(self.input_dim, self.input_dim),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(self.input_dim, self.input_dim),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(self.input_dim, 2),
        nn.Sigmoid()
    )


  def forward(self, x):
        #x = x.squeeze()
        x = torch.flatten(x,start_dim=1)
        assert x.shape[1]==1024*3
                
        out = self.layers(x)
        
        return 32.0*out



In [44]:
import torch
import numpy as np
import pylab as plt
from skimage import filters
import math
side = 32
a = torch.zeros(1024,32,32,3)
b = torch.zeros(1024,2)
for i in range(32):
  for j in range(32):
    a[:,i,j,0] = i
    a[:,i,j,1] = j

for i in range(1024):
  for j in range(32):
    for k in range(32):
      a[i,j,k,2] = 1
      b[i,0]=j
      b[i,1]=k      


def point_matrix():
    length = 32**2

class PointDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.length = side**2
        self.values = a
        self.gt = b

    def __len__(self):
        return self.length


    def __getitem__(self, idx):
      
        return a[idx,:,:,:], b[idx,:]
    
    @staticmethod
    def displayCanvas(title,dataset, model):
        #model.setBatchSize(batch_size = 1)
        for i in range(36):
            sample, labels = dataset[i]
            plt.subplot(6,6,i+1)
            plot_all(sample = sample,model=model, labels = labels)
            plt.axis('off')
        plt.savefig(title,dpi=600)

dataset = PointDataset()
#PointDataset.displayCanvas('donut.png',dataset, model = None)


In [45]:
from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

mini_batch = 64
dataset = PointDataset()
loader_train = data.DataLoader(
    dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=dataset),
    num_workers=4)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [46]:
model = MLP().cuda()

optimizer = torch.optim.Adam(model.parameters(),lr = 0.000001, betas = (.9,.999))


for i in range(10):
  for x,y in loader_train:
    optimizer.zero_grad()
    x = x.cuda()
    y = y.cuda()
    out = model(x)
    loss = torch.mean((y-out)**2)
    loss.backward()
    optimizer.step()
    print(loss)



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


tensor(473.5018, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(525.7179, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(386.0582, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(401.6327, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(296.8934, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(322.1432, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(352.5598, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(279.3042, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(304.2860, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(242.0604, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(259.5600, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(234.8600, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(205.2250, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(153.8866, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(194.7016, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(182.2358, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(132.9231, device='cuda:0', grad_fn=<MeanBackward0

In [40]:
model.layers[6].bias


Parameter containing:
tensor([ 0.0177, -0.0090], device='cuda:0', requires_grad=True)

This experiment (some results unseen) demonstrated that the uber paper helped in this instance. also that maximum accuracy was 1 pixel in some systematic error way. i checked the bias terms and saw nothing concerning. 